<a href="https://colab.research.google.com/github/gjcarranza/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [4]:
totalvotes_df = df.filter(df.total_votes >= 20)
totalvotes_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   23282003|R2SHXRL6SL1GC9|B010FP0WRU|     974013859|#392: The EP Coll...|           Music|          3|           25|         26|   N|                Y|Up it by a half-s...|A solid collectio...| 2015-08-31|
|         US|   21715314|R2ZC033X86YOY8|B00ZYBH6M0|     408426475|  Live in Tokyo 1975|           Music|          5|    

In [6]:
helpfulv_df = totalvotes_df.filter("helpful_votes/total_votes>=.50")
helpfulv_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   23282003|R2SHXRL6SL1GC9|B010FP0WRU|     974013859|#392: The EP Coll...|           Music|          3|           25|         26|   N|                Y|Up it by a half-s...|A solid collectio...| 2015-08-31|
|         US|   21715314|R2ZC033X86YOY8|B00ZYBH6M0|     408426475|  Live in Tokyo 1975|           Music|          5|    

In [7]:
paid_df = helpfulv_df.filter(helpfulv_df.vine == 'Y')
paid_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45725380| R84VUCDBCI29U|B0020H46LK|     630657414|        Losing Sleep|           Music|          3|           18|         28|   Y|                N|Suitable for Mass...|Do you still get ...| 2009-07-31|
|         US|   53081008| R4V3ICFDTIDIF|B001SMC91M|     609338227|Tell 'Em What You...|           Music|          4|    

In [8]:
npaid_df = helpfulv_df.filter(helpfulv_df.vine == 'N')
npaid_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   23282003|R2SHXRL6SL1GC9|B010FP0WRU|     974013859|#392: The EP Coll...|           Music|          3|           25|         26|   N|                Y|Up it by a half-s...|A solid collectio...| 2015-08-31|
|         US|   21715314|R2ZC033X86YOY8|B00ZYBH6M0|     408426475|  Live in Tokyo 1975|           Music|          5|    

In [9]:
total_review_count = df.count()
total_review_count

4751577

In [10]:
totalfive_df=df.filter(df["star_rating"] == '5').count()
totalfive_df

3290473

In [11]:
paidfivestar_df=paid_df.filter(paid_df.star_rating == 5).count()
paidfivestar_df

0

In [12]:
npaidfive_df=npaid_df.filter(npaid_df.star_rating == 5).count()
npaidfive_df

67580

In [18]:
# percentage of paid 5 star reviews
paidfive=(paidfivestar_df/total_review_count)
paidfive

0.0

In [19]:
# percentage of non-paid 5 star reviews
npaidfive=(npaidfive_df/total_review_count)
npaidfive

0.01422264650241383